In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.mixture import BayesianGaussianMixture
import lightgbm as lgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2022/data.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2022/sample_submission.csv")

In [ ]:
data.head()

In [ ]:
sub.head()

In [ ]:
data.shape

In [ ]:
sub.shape

**EDA**

In [ ]:
data.describe()

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(data.corr(),annot=True,fmt=".2f")

In [ ]:
plt.figure(figsize=(25,20))
for i,column in enumerate(list(data.columns),1):
    plt.subplot(5,6,i)
    sns.histplot(x=column,data=data.sample(1000),kde=True)

**Feature Engineering**

In [ ]:
data = data.drop(columns="id")

In [ ]:
cols = list(data.columns)

In [ ]:
data_scaled = PowerTransformer(method="yeo-johnson").fit_transform(data)

In [ ]:
data_scaled = pd.DataFrame(data_scaled,columns=cols)
data_scaled.head()

In [ ]:
plt.figure(figsize=(25,20))
for i,column in enumerate(list(data_scaled.columns),1):
    plt.subplot(5,6,i)
    sns.histplot(x=column,data=data_scaled.sample(1000),kde=True)

**Model**

In [ ]:
vbgmm = BayesianGaussianMixture(n_components=7,covariance_type='full',random_state=1)
preds = vbgmm.fit_predict(data_scaled)

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(15,6))
for i in range(vbgmm.means_.shape[0]):
    plt.scatter(np.arange(data_scaled.shape[1]), vbgmm.means_[i])
plt.xticks(ticks=np.arange(data_scaled.shape[1]), labels=cols)
plt.show()

In [ ]:
important_cols = ['f_07', 'f_08', 'f_09', 'f_10', 'f_11', 'f_12', 'f_13', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_27', 'f_28', ]

In [ ]:
preds = vbgmm.fit_predict(data_scaled[important_cols])

In [ ]:
pp = vbgmm.predict_proba(data_scaled[important_cols])

In [ ]:
data_scaled[[f'predict_proba_{i}' for i in range(7)]]=pp 
data_scaled['predict_proba']=np.max(pp,axis=1)
data_scaled['predict']=np.argmax(pp,axis=1)
data_scaled

In [ ]:
trn_indx=np.array([])
for n in range(7):
    indx=data_scaled[(data_scaled.predict==n) & (data_scaled.predict_proba > 0.80)].index 
    trn_indx = np.concatenate((trn_indx, indx))
    
X = data_scaled.loc[trn_indx][important_cols]
y = data_scaled.loc[trn_indx]['predict']

In [ ]:
params = {'learning_rate': 0.07,'objective': 'multiclass','boosting': 'gbdt','verbosity': -1,'n_jobs': -1, 'num_classes':7} 

lgbm_predict_proba = 0  
classif_scores = []

kf = StratifiedKFold(5, shuffle=True, random_state = 1)
for fold, (trn, val) in enumerate(kf.split(X,y)):   

    X_trn = lgb.Dataset(X.iloc[trn], y.iloc[trn], feature_name = important_cols)
    X_val = lgb.Dataset(X.iloc[val], y.iloc[val], feature_name = important_cols)
    
    model = lgb.train(params = params, 
                train_set = X_trn, valid_sets =  X_val, 
                num_boost_round = 5000, 
                callbacks = [ lgb.early_stopping(stopping_rounds=100, verbose=True), lgb.log_evaluation(period=200)])  
    
    y_pred_proba = model.predict(X.iloc[val])
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    s = (balanced_accuracy_score(y.iloc[val], y_pred),
        roc_auc_score(y.iloc[val], y_pred_proba, average="weighted", multi_class="ovo"))
    classif_scores.append(s)

    lgbm_predict_proba += model.predict(data_scaled[important_cols]) / 5
    


**Submission**

In [ ]:
sub["Predicted"] = np.argmax(lgbm_predict_proba,axis=1)

In [ ]:
sub.to_csv('submission.csv', index=False)